## Set Global Seed 

In [1]:
import os
import random
import numpy as np
import torch
import transformers

def set_all_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed = 260615
set_all_seeds(seed)

print("The global seed " + str(seed))

The global seed 260615


## Hyperparameters

In [2]:
# LANGUAGE

_LANGUAGE_         = 'en'
_DATASET_          = '2021'

In [3]:
# MODEL CLASSIFICATION

_PRETRAINED_LM_    = 'vinai/bertweet-base'
_PREPROCESS_TEXT_  = True
_TWEET_BATCH_SIZE_ = 5
_ADAPTER_CONFIG_   = transformers.ParallelConfig(reduction_factor = 256)
_MAX_SEQ_LEN_      = 128

In [4]:
# TRAIN

_OUTPUT_DIR_       = 'checkPointsFFT'
_LOGGING_STEPS_    = 50
_NUM_AUTHORS_      = 64
_K_FOLD_CV_        = 5
_NO_GPUS_          = 1
_BATCH_SIZE_       = int(32 / _NO_GPUS_)
_EPOCHS_           = 10
_LEARNING_RATE_    = 1e-5

# PREDICTIONS

_PRED_DIR_         = 'FFTTestGeneralized'

## Other parameters

In [5]:
# LABEL DICTONARIES -----------------------------------------------------------------------

# 2017

gender_dict    = {'female': 0, 'male':   1}
varietyEN_dict = {'australia': 0, 'canada': 1, 'great britain': 2, 'ireland': 3, 'new zealand': 4, 'united states': 5}
varietyES_dict = {'argentina': 0, 'chile': 1, 'colombia': 2, 'mexico': 3, 'peru': 4, 'spain': 5, 'venezuela': 6}  

genderEN_hyp  = {0: 'I’m a female', 1: 'I’m a male'}
genderES_hyp  = {0: 'Mi nombre es María', 1: 'Mi nombre es José'}

# 2019

bots_dict  = {'human': 0, 'bot': 1}
botsEN_hyp = {0: 'This is a text from a person', 1: 'This is a text from a machine'}
botsES_hyp = {0: 'Humano', 1: 'Bot'}

# 2020 

fakeNews_dict  = {'0': 0, '1': 1}
fakeNewsEN_hyp = {0: 'This author is a normal user', 1: 'This author spreads fake news'}
fakeNewsES_hyp = {0: 'Este autor es un usuario normal', 1: 'Este autor publica noticias falsas'}

# 2021

hateSpeech_dict  = {'0': 0, '1': 1}
hateSpeechEN_hyp = {0: 'This text does not contain hate speech', 1: 'This text expresses prejudice and hate speech'}
hateSpeechES_hyp = {0: 'Este texto es moderado, respetuoso, cortés y civilizado', 1: 'Este texto expresa odio o prejuicios'}

In [6]:
# SET LANGUAGE DICTIONARIES --------------------------------------------------

if _LANGUAGE_ == 'en':
    gender_hyp     = genderEN_hyp
    variety_dict   = varietyEN_dict
    fakeNews_hyp   = fakeNewsEN_hyp
    hateSpeech_hyp = hateSpeechEN_hyp
    bots_hyp       = botsEN_hyp 

elif _LANGUAGE_ == 'es':
    gender_hyp     = genderES_hyp
    variety_dict   = varietyES_dict
    fakeNews_hyp   = fakeNewsES_hyp
    hateSpeech_hyp = hateSpeechES_hyp
    bots_hyp       = botsES_hyp
    
    
# SET LANGUAGE AND DATASET PARAMETERS ----------------------------------------
    
if   _DATASET_ == '2017':
    label_idx  = 1
    class_dict = gender_dict
    label_name = 'gender'
    label_hyp  = gender_hyp
    
elif _DATASET_ == '2019':
    label_idx  = 1
    class_dict = bots_dict
    label_name = 'bots'
    label_hyp  = bots_hyp
    
elif _DATASET_ == '2020':
    label_idx  = 1
    class_dict = fakeNews_dict
    label_name = 'fakeNews'
    label_hyp  = fakeNews_hyp
    
elif _DATASET_ == '2021':
    label_idx  = 1
    class_dict = hateSpeech_dict
    label_name = 'hateSpeech'
    label_hyp  = hateSpeech_hyp
    

In [7]:
# SET LANGUAGE TOKENIZER

from transformers import AutoTokenizer, PretrainedConfig

tokenizer = AutoTokenizer.from_pretrained(_PRETRAINED_LM_)
vocab = tokenizer.get_vocab()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Datasets

In [8]:
# GET AUTHORS AND LABELS -----------------------------------------------------

from tools.DataLoaders import BasePAN

baseTrain  = BasePAN(Dir        = 'data/' + _DATASET_,
                     split      = 'train',
                     language   = _LANGUAGE_,
                     label_idx  = label_idx,
                     class_dict = class_dict,
                     label_name = label_name)
"""
baseTest   = BasePAN(Dir        = 'data/' + _DATASET_,
                     split      = 'test',
                     language   = _LANGUAGE_,
                     label_idx  = label_idx,
                     class_dict = class_dict,
                     label_name = label_name)
"""

"\nbaseTest   = BasePAN(Dir        = 'data/' + _DATASET_,\n                     split      = 'test',\n                     language   = _LANGUAGE_,\n                     label_idx  = label_idx,\n                     class_dict = class_dict,\n                     label_name = label_name)\n"

In [9]:
# GET K-FOLD SPLITS -----------------------------------------------------

crossVal_splits = baseTrain.cross_val(_K_FOLD_CV_, _NUM_AUTHORS_)

crossVal_splits[0][0]

['98c59ff3e2cfc56b96ee3c49b3d46296',
 'aa917a8c5a4420b024274672667c7dc1',
 '86a7f84c2dd126dac46270b6c912952c',
 '9cff4936f8479d53fcbb63f2524c5ad8',
 '30be488aa93e8000aadb952a9cd5143c',
 'a8e2397021acef98cc32729cbda96910',
 '3770a07b212c1096c26e5a1f1556fbd1',
 'b8924a54bb6043c56969e20a328b76b3',
 '258ba7b57bc38e4987f9f3cf23700ece',
 '26644d1348fc1122e8c5ef45d6bc84fa',
 '54f81e27af90ed7c1c9409c332f0ca37',
 '58584745632b5367da1c7a9af746222b',
 '7f269488a6576c9dc21085c1e2854142',
 '6711ef348ffcb3e45d2957396a4c8026',
 '3df768933d03108ea4c6583d49c85c46',
 '365eb1e3abc5cd5394fec8fc162bfbc5',
 'b496caf332cb0ba97d2acefc44f153ac',
 'fdef657f264ca50bc7b21574b24f82ab',
 '76e152a7732922e7a6da39880486107f',
 '4253c341c1069eded30b6efd2df89ddc',
 '4f496db1408c402eb21d29e536667205',
 '4ae4ddc8cb2774c92398e3102c3da5b2',
 'f3eecd0eedab3b77558d93b1b92579a4',
 '4a1baf66990e0e540effd01f4b105f44',
 '748c4b31797d62bcce99de35a681b484',
 'a887ec85088a87e550015e2770a6e309',
 '9d58d6313bfb2fba9e1e45bb9d65cf0b',
 

In [10]:
# GET TWEETS -----------------------------------------------------

baseTrain.get_all_data(_TWEET_BATCH_SIZE_, tokenizer, _MAX_SEQ_LEN_, _PREPROCESS_TEXT_)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 8000



## Training

In [11]:
from transformers import TrainingArguments

samples = 2 * _NUM_AUTHORS_ * int(100 / _TWEET_BATCH_SIZE_)
_LOGGING_STEPS_ = int(samples / _BATCH_SIZE_)

training_args = TrainingArguments(
    learning_rate               = _LEARNING_RATE_,
    num_train_epochs            = _EPOCHS_,
    per_device_train_batch_size = _BATCH_SIZE_,
    per_device_eval_batch_size  = 200,
    logging_steps               = _LOGGING_STEPS_,
    output_dir                  = _OUTPUT_DIR_,
    save_total_limit            = 10,
    overwrite_output_dir        = True,
    remove_unused_columns       = False,
)

In [12]:
from transformers import AutoModelForSequenceClassification
from tools.DataLoaders import DatasetCrossVal
from transformers import Trainer
from tools.Testing import compute_author_predictions
from sklearn.metrics import f1_score, classification_report
import pickle


# train

task = label_name

f1s_soft = []
f1s_hard = []

for split in range( _K_FOLD_CV_ ):
    
    # loaders for current split ------------------------------------------
    
    authors_train, authors_val = crossVal_splits[split]
    
    Train = DatasetCrossVal(baseTrain, authors_train, task)
    Val   = DatasetCrossVal(baseTrain, authors_val  , task)
    
    
    # initialize model ---------------------------------------------------
    
    model = AutoModelForSequenceClassification.from_pretrained(_PRETRAINED_LM_, num_labels = len(class_dict))
    
    
    # create trainer and train -------------------------------------------
        
    trainer = Trainer(
        model           = model,
        args            = training_args,
        train_dataset   = Train,
    )
    trainer.args._n_gpu = _NO_GPUS_

    trainer.train()
    
    
    # get predictions ----------------------------------------------------
    
    results            = trainer.predict(Val)
    author_predictions = compute_author_predictions(Val, results.predictions, task, len(class_dict))
    
    
    # report metrics -----------------------------------------------------
    
    report = {'soft': classification_report(author_predictions['true'], author_predictions['pred_soft'], digits=4), 
               'hard': classification_report(author_predictions['true'], author_predictions['pred_hard'], digits=4)}

    f1s_soft.append( f1_score(author_predictions['true'], author_predictions['pred_soft'], average = 'macro') )
    f1s_hard.append( f1_score(author_predictions['true'], author_predictions['pred_hard'], average = 'macro') )

    print("Results with split " + str(split + 1) + ":\n")
    print("soft voting:\n", report['soft'], '\n')
    print("hard voting:\n", report['hard'])
     
    
    # save predictions ----------------------------------------------------
    
    DIR = 'results/' + _DATASET_ + '/' + _LANGUAGE_ + '/' + _PRED_DIR_ + '/' + str(_NUM_AUTHORS_) + '_authors/split_' + str(split + 1) + '/'
    if not os.path.exists(DIR):
        os.makedirs(DIR)

    with open(DIR + 'predictions.pickle', 'wb') as f:
        pickle.dump(author_predictions, f)

    with open(DIR + 'report.txt', 'w') as f:
        f.write("soft voting:\n" + report['soft'] + '\n\n')
        f.write("hard voting:\n" + report['hard'])
    


Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

Step,Training Loss
80,0.684100
160,0.640300
240,0.585000
320,0.555000
400,0.507800
480,0.495700
560,0.433900
640,0.414900
720,0.349600
800,0.353800


Saving model checkpoint to checkPointsFFT/checkpoint-500
Configuration saved in checkPointsFFT/checkpoint-500/config.json
Model weights saved in checkPointsFFT/checkpoint-500/pytorch_model.bin
Saving model checkpoint to checkPointsFFT/checkpoint-1000
Configuration saved in checkPointsFFT/checkpoint-1000/config.json
Model weights saved in checkPointsFFT/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [checkPointsFFT/checkpoint-2000] due to args.save_total_limit
Saving model checkpoint to checkPointsFFT/checkpoint-1500
Configuration saved in checkPointsFFT/checkpoint-1500/config.json
Model weights saved in checkPointsFFT/checkpoint-1500/pytorch_model.bin
Deleting older checkpoint [checkPointsFFT/checkpoint-2500] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 1600
  Batch size = 200


100%|████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 347.60it/s]


Results with split 1:

soft voting:
               precision    recall  f1-score   support

           0     0.7500    0.6000    0.6667        20
           1     0.6667    0.8000    0.7273        20

    accuracy                         0.7000        40
   macro avg     0.7083    0.7000    0.6970        40
weighted avg     0.7083    0.7000    0.6970        40
 

hard voting:
               precision    recall  f1-score   support

           0     0.8000    0.6000    0.6857        20
           1     0.6800    0.8500    0.7556        20

    accuracy                         0.7250        40
   macro avg     0.7400    0.7250    0.7206        40
weighted avg     0.7400    0.7250    0.7206        40



...

loading configuration file https://huggingface.co/vinai/bertweet-base/resolve/main/config.json from cache at /001/usuarios/isaac.bribiesca/.cache/huggingface/transformers/356366feedcea0917e30f7f235e1e062ffc2d28138445d5672a184be756c8686.a2b6026e688d1b19cebc0981d8f3a5b1668eabfda55b2c42049d5eac0bc8cb2d
Model config RobertaConfig {
  "_name_or_path": "vinai/bertweet-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertweetTokenizer",
  "transformers_version": "4.21.3",
  "type_vocab

Step,Training Loss
80,0.673900
160,0.624800
240,0.579800
320,0.543500
400,0.488700
480,0.486000
560,0.417000
640,0.425600
720,0.364400
800,0.324600


Saving model checkpoint to checkPointsFFT/checkpoint-500
Configuration saved in checkPointsFFT/checkpoint-500/config.json
Model weights saved in checkPointsFFT/checkpoint-500/pytorch_model.bin
Saving model checkpoint to checkPointsFFT/checkpoint-1000
Configuration saved in checkPointsFFT/checkpoint-1000/config.json
Model weights saved in checkPointsFFT/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to checkPointsFFT/checkpoint-1500
Configuration saved in checkPointsFFT/checkpoint-1500/config.json
Model weights saved in checkPointsFFT/checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 1600
  Batch size = 200


100%|████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 760.28it/s]

Results with split 5:

soft voting:
               precision    recall  f1-score   support

           0     0.6500    0.6500    0.6500        20
           1     0.6500    0.6500    0.6500        20

    accuracy                         0.6500        40
   macro avg     0.6500    0.6500    0.6500        40
weighted avg     0.6500    0.6500    0.6500        40
 

hard voting:
               precision    recall  f1-score   support

           0     0.6500    0.6500    0.6500        20
           1     0.6500    0.6500    0.6500        20

    accuracy                         0.6500        40
   macro avg     0.6500    0.6500    0.6500        40
weighted avg     0.6500    0.6500    0.6500        40



In [13]:
# report statistics

print('Soft results: ', f1s_soft)
print('\nHard results: ', f1s_hard)

f1s_soft = np.array(f1s_soft)
f1s_hard = np.array(f1s_hard)

FewShot_Results = {'soft': [f1s_soft.mean(), f1s_soft.std()], 'hard': [f1s_hard.mean(), f1s_hard.std()]}

print('\n\nSoft statistics: ')
print('\t[avg, std]:', FewShot_Results['soft'])

print('\nHard statistics: ')
print('\t[avg, std]:', FewShot_Results['hard'])

Soft results:  [0.6969696969696968, 0.7, 0.5523370638578011, 0.7, 0.65]

Hard results:  [0.7206349206349207, 0.6731615336266499, 0.5523370638578011, 0.7, 0.65]


Soft statistics: 
	[avg, std]: [0.6598613521654996, 0.05702275019092]

Hard statistics: 
	[avg, std]: [0.6592267036238744, 0.058545943412231585]


## Training and Testing

In [11]:
from tools.DataLoaders import DatasetPAN

baseTest.get_all_data(_TWEET_BATCH_SIZE_, tokenizer, _MAX_SEQ_LEN_, _PREPROCESS_TEXT_)

Test = DatasetPAN(baseTest, label_name)


Reading data...
    Done
Preprocessing text...
    Done
Tokenizing...
    Done
Merging data...
    Done

Total Instances: 4000



In [12]:
from transformers import TrainingArguments

samples = 2 * _NUM_AUTHORS_ * int(100 / _TWEET_BATCH_SIZE_)
_LOGGING_STEPS_ = int(samples / _BATCH_SIZE_)

training_args = TrainingArguments(
    learning_rate               = _LEARNING_RATE_,
    num_train_epochs            = _EPOCHS_,
    per_device_train_batch_size = _BATCH_SIZE_,
    per_device_eval_batch_size  = 200,
    logging_steps               = _LOGGING_STEPS_,
    output_dir                  = _OUTPUT_DIR_,
    save_total_limit            = 10,
    overwrite_output_dir        = True,
    remove_unused_columns       = False,
)

In [ ]:
from transformers import AutoModelForSequenceClassification
from tools.DataLoaders import DatasetCrossVal
from transformers import Trainer
from tools.Testing import compute_author_predictions
from sklearn.metrics import f1_score, classification_report
import pickle


# train

task = label_name

f1s_soft = []
f1s_hard = []

for split in range( _K_FOLD_CV_ ):
    
    # loaders for current split ------------------------------------------
    
    authors_train, authors_val = crossVal_splits[split]
    
    Train = DatasetCrossVal(baseTrain, authors_train, task)
    Val   = DatasetCrossVal(baseTrain, authors_val  , task)
    
    
    # initialize model ---------------------------------------------------
    
    model = AutoModelForSequenceClassification.from_pretrained(_PRETRAINED_LM_, num_labels = len(class_dict))
    
    
    # create trainer and train -------------------------------------------
        
    trainer = Trainer(
        model           = model,
        args            = training_args,
        train_dataset   = Train,
    )
    trainer.args._n_gpu = _NO_GPUS_

    trainer.train()
    
    
    # get predictions ----------------------------------------------------
    
    results            = trainer.predict(Test)
    author_predictions = compute_author_predictions(baseTest, results.predictions, task, len(class_dict))
    
    
    # report metrics -----------------------------------------------------
    
    report = {'soft': classification_report(author_predictions['true'], author_predictions['pred_soft'], digits=4), 
               'hard': classification_report(author_predictions['true'], author_predictions['pred_hard'], digits=4)}

    f1s_soft.append( f1_score(author_predictions['true'], author_predictions['pred_soft'], average = 'macro') )
    f1s_hard.append( f1_score(author_predictions['true'], author_predictions['pred_hard'], average = 'macro') )

    print("Results with split " + str(split + 1) + ":\n")
    print("soft voting:\n", report['soft'], '\n')
    print("hard voting:\n", report['hard'])
     
    
    # save predictions ----------------------------------------------------
    
    DIR = 'results/' + _DATASET_ + '/' + _LANGUAGE_ + '/' + _PRED_DIR_ + '/' + str(_NUM_AUTHORS_) + '_authors/test_split_' + str(split + 1) + '/'
    if not os.path.exists(DIR):
        os.makedirs(DIR)

    with open(DIR + 'predictions.pickle', 'wb') as f:
        pickle.dump(author_predictions, f)

    with open(DIR + 'report.txt', 'w') as f:
        f.write("soft voting:\n" + report['soft'] + '\n\n')
        f.write("hard voting:\n" + report['hard'])
    


Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: 

Step,Training Loss
80,0.597100
160,0.415400
240,0.318800
320,0.250600
400,0.189700
480,0.165900
560,0.111700
640,0.097700
720,0.073200
800,0.066600


Saving model checkpoint to checkPointsFFT/checkpoint-500
Configuration saved in checkPointsFFT/checkpoint-500/config.json
Model weights saved in checkPointsFFT/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Prediction *****
  Num examples = 4000
  Batch size = 200


100%|██████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 488.29it/s]


Results with split 1:

soft voting:
               precision    recall  f1-score   support

           0     0.6633    0.6500    0.6566       100
           1     0.6569    0.6700    0.6634       100

    accuracy                         0.6600       200
   macro avg     0.6601    0.6600    0.6600       200
weighted avg     0.6601    0.6600    0.6600       200
 

hard voting:
               precision    recall  f1-score   support

           0     0.6535    0.6600    0.6567       100
           1     0.6566    0.6500    0.6533       100

    accuracy                         0.6550       200
   macro avg     0.6550    0.6550    0.6550       200
weighted avg     0.6550    0.6550    0.6550       200



...

100%|██████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 581.08it/s]


Results with split 4:

soft voting:
               precision    recall  f1-score   support

           0     0.6058    0.8300    0.7004       100
           1     0.7302    0.4600    0.5644       100

    accuracy                         0.6450       200
   macro avg     0.6680    0.6450    0.6324       200
weighted avg     0.6680    0.6450    0.6324       200
 

hard voting:
               precision    recall  f1-score   support

           0     0.6127    0.8700    0.7190       100
           1     0.7759    0.4500    0.5696       100

    accuracy                         0.6600       200
   macro avg     0.6943    0.6600    0.6443       200
weighted avg     0.6943    0.6600    0.6443       200



loading configuration file https://huggingface.co/vinai/bertweet-base/resolve/main/config.json from cache at /001/usuarios/isaac.bribiesca/.cache/huggingface/transformers/356366feedcea0917e30f7f235e1e062ffc2d28138445d5672a184be756c8686.a2b6026e688d1b19cebc0981d8f3a5b1668eabfda55b2c42049d5eac0bc8cb2d
Model config RobertaConfig {
  "_name_or_path": "vinai/bertweet-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertweetTokenizer",
  "transformers_version": "4.21.3",
  "type_vocab

Step,Training Loss
80,0.606700
160,0.438600
240,0.324800
320,0.262500
400,0.212600
480,0.147000
560,0.128700
640,0.099800
720,0.086900


Saving model checkpoint to checkPointsFFT/checkpoint-500
Configuration saved in checkPointsFFT/checkpoint-500/config.json
Model weights saved in checkPointsFFT/checkpoint-500/pytorch_model.bin


In [ ]:
# report statistics

print('Soft results: ', f1s_soft)
print('\nHard results: ', f1s_hard)

f1s_soft = np.array(f1s_soft)
f1s_hard = np.array(f1s_hard)

FewShot_Results = {'soft': [f1s_soft.mean(), f1s_soft.std()], 'hard': [f1s_hard.mean(), f1s_hard.std()]}

print('\n\nSoft statistics: ')
print('\t[avg, std]:', FewShot_Results['soft'])

print('\nHard statistics: ')
print('\t[avg, std]:', FewShot_Results['hard'])